# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: lora 
* Model: bert
* Evaluation approach: Trainer.evaluate()
* Fine-tuning dataset: dair-ai/emotion 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
from transformers import AutoModelForSequenceClassification

# Load pretrained model and adapt labels
base_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased",
    num_labels=6,
    id2label={0: "sadness", 1: "joy", 2: "love", 3: "anger", 4: "fear", 5: "surprise"},  
    label2id={"sadness": 0, "joy": 1, "love": 2, "anger": 3, "fear": 4, "surprise": 5},)
# Freeze the embedding parameters - Thanks to Merhat for the hint of not freezing all parameters
for param in base_model.distilbert.embeddings.parameters():
    param.requires_grad = True
    
# Add classifier
base_model.classifier
print(base_model)


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [2]:
# Save the base model so it can be loaded later for evaluation and comparison
base_model.save_pretrained("./models/base")

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Get dataset splits
splits = ["train", "validation", "test"]
ds = {split: ds for split, ds in zip(splits, load_dataset("dair-ai/emotion", split=splits))}

# Print dataset overview
print(ds)

# Limit dataset size for training time in this exercise
ds["train"] = ds["train"].shuffle(seed=21).select(range(6000))
ds["validation"] = ds["validation"].shuffle(seed=22).select(range(2000))
ds["test"] = ds["train"].shuffle(seed=23).select(range(1000))

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(data):
    # Return tokenized data
    encoded = tokenizer(data['text'], padding='max_length', truncation=True)
    return encoded

# Tokenize all dataset splits
tokenized_ds = {}
for split in ds.keys():
    tokenized_ds[split] = ds[split].map(preprocess_function, batched=True)
   
# Print an example
print(tokenized_ds["train"][0]["input_ids"])

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'train': Dataset({
    features: ['text', 'label'],
    num_rows: 16000
}), 'validation': Dataset({
    features: ['text', 'label'],
    num_rows: 2000
}), 'test': Dataset({
    features: ['text', 'label'],
    num_rows: 2000
})}


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

[101, 1045, 2123, 1056, 2514, 2061, 6091, 2725, 2047, 2477, 4902, 1045, 2031, 2062, 1997, 2019, 2023, 2003, 2054, 1045, 2031, 2000, 2079, 1998, 1045, 2097, 2079, 2009, 2828, 1997, 7729, 2738, 2084, 2019, 1045, 2428, 3246, 1045, 2123, 2102, 11224, 2039, 2828, 1997, 7729, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [4]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

# Setup Trainer for evaluation using the test split
trainer = Trainer(
    model=base_model,
    args=TrainingArguments(
        output_dir="./data/base_evaluation",
        learning_rate=1e-2,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=5,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# Evaluate the base model
eval_base = trainer.evaluate()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [5]:
eval_base

{'eval_loss': 1.8028442859649658,
 'eval_accuracy': 0.034,
 'eval_runtime': 15.9422,
 'eval_samples_per_second': 62.726,
 'eval_steps_per_second': 2.007}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [6]:
from peft import LoraConfig, TaskType, get_peft_model

# Target value and key projection with lora - Thanks Merhat for the hint
peft_config_vk = LoraConfig(task_type='SEQ_CLS',target_modules=['v_lin', 'k_lin'], r=8, lora_alpha=32, lora_dropout=0.1)

# Target query projection with lora for comparison
peft_config_q = LoraConfig(task_type='SEQ_CLS',target_modules=['q_lin'], r=8, lora_alpha=32, lora_dropout=0.1)

peft_model_vk = get_peft_model(base_model, peft_config_vk)
peft_model_vk.print_trainable_parameters()

peft_model_q = get_peft_model(base_model, peft_config_q)
peft_model_q.print_trainable_parameters()

trainable params: 1,337,868 || all params: 67,700,748 || trainable%: 1.9761495101944813
trainable params: 221,184 || all params: 67,774,476 || trainable%: 0.32635294738390896


In [7]:
# Setup Trainer for training using the train and validation split
training_args = TrainingArguments(
    output_dir="./data/peft_vk_training",
    learning_rate=1e-3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

peft_trainer_vk = Trainer(
    model=peft_model_vk,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

In [8]:
peft_trainer_vk.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.928085,0.724000
2,No log,0.789107,0.783000


TrainOutput(global_step=376, training_loss=1.063731092087766, metrics={'train_runtime': 537.2756, 'train_samples_per_second': 22.335, 'train_steps_per_second': 0.7, 'total_flos': 1619817578496000.0, 'train_loss': 1.063731092087766, 'epoch': 2.0})

In [27]:
# Setup Trainer for evaluation using the test split
training_args = TrainingArguments(
    output_dir="./data/peft_vk_evaluation_final",
    learning_rate=1e-3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="no",
)
peft_trainer_vk2 = Trainer(
    model=peft_model_vk,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)
peft_trainer_vk2.evaluate()


{'eval_loss': 0.3937908709049225,
 'eval_accuracy': 0.92,
 'eval_runtime': 18.257,
 'eval_samples_per_second': 54.773,
 'eval_steps_per_second': 1.753}

In [65]:
peft_model_vk.save_pretrained("./models/peft_vk_model")

In [10]:
# Setup Trainer for training using the train and validation split
training_args = TrainingArguments(
    output_dir="./data/peft_q_training",
    learning_rate=1e-3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

peft_trainer_q = Trainer(
    model=peft_model_q,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

In [11]:
peft_trainer_q.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.573518,0.841000
2,No log,0.454280,0.905500


TrainOutput(global_step=376, training_loss=0.5982268313144116, metrics={'train_runtime': 545.594, 'train_samples_per_second': 21.994, 'train_steps_per_second': 0.689, 'total_flos': 1619817578496000.0, 'train_loss': 0.5982268313144116, 'epoch': 2.0})

In [63]:
peft_model_q.save_pretrained("./models/peft_q_model")

NameError: name 'peft_model' is not defined

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [13]:
from transformers import AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer, TrainingArguments

# Load the base model
loaded_base_model = AutoModelForSequenceClassification.from_pretrained("./models/base")

# Setup Trainer for evaluation using the test split
training_args = TrainingArguments(
    output_dir="./data/base_evaluation_final",
    learning_rate=1e-3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="no",
)
base_trainer = Trainer(
    model=loaded_base_model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

In [14]:
# Evaluate the loaded base model
eval_base2 = base_trainer.evaluate()
eval_base2

{'eval_loss': 1.8028442859649658,
 'eval_accuracy': 0.034,
 'eval_runtime': 18.3037,
 'eval_samples_per_second': 54.634,
 'eval_steps_per_second': 1.748}

In [75]:
from peft import PeftConfig, AutoPeftModelForSequenceClassification

# Load the trained peft model
loaded_peft_model_vk = AutoPeftModelForSequenceClassification.from_pretrained("./models/peft_vk_model", num_labels=6,
    id2label={0: "sadness", 1: "joy", 2: "love", 3: "anger", 4: "fear", 5: "surprise"},  
    label2id={"sadness": 0, "joy": 1, "love": 2, "anger": 3, "fear": 4, "surprise": 5},)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [76]:
# Setup Trainer for evaluation using the test split
training_args = TrainingArguments(
    output_dir="./data/peft_vk_evaluation_final",
    learning_rate=1e-3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="no",
)
peft_trainer_vk = Trainer(
    model=loaded_peft_model_vk,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)


In [77]:
# Evaluation of PEFT model
eval_peft_vk = peft_trainer_vk.evaluate()
eval_peft_vk

{'eval_loss': 1.7993906736373901,
 'eval_accuracy': 0.034,
 'eval_runtime': 18.0723,
 'eval_samples_per_second': 55.333,
 'eval_steps_per_second': 1.771}

In [79]:
# Try a different PEFT config for comparison
loaded_peft_model_q = AutoPeftModelForSequenceClassification.from_pretrained("./models/peft_q_model",
                                                                              num_labels=6,
    id2label={0: "sadness", 1: "joy", 2: "love", 3: "anger", 4: "fear", 5: "surprise"},  
    label2id={"sadness": 0, "joy": 1, "love": 2, "anger": 3, "fear": 4, "surprise": 5},)
# Setup Trainer for evaluation using the test split
training_args = TrainingArguments(
    output_dir="./data/peft_q_evaluation_final",
    learning_rate=1e-3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="no",
)
peft_trainer_q = Trainer(
    model=loaded_peft_model_q,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [80]:
# Evaluation of PEFT model
eval_peft_q = peft_trainer_q.evaluate()
eval_peft_q

{'eval_loss': 1.7993906736373901,
 'eval_accuracy': 0.034,
 'eval_runtime': 18.876,
 'eval_samples_per_second': 52.977,
 'eval_steps_per_second': 1.695}

In [81]:
# Compare the results
print("eval_base2\n", eval_base2)
print("eval_peft_vk\n", eval_peft_vk)
print("eval_peft_q\n", eval_peft_q)

print("Base accuracy: ", eval_base2['eval_accuracy'], "\nPEFT_vk accuracy: ", eval_peft_vk['eval_accuracy'], "\nPEFT_q accuracy: ", eval_peft_q['eval_accuracy'])

eval_base2
 {'eval_loss': 1.8028442859649658, 'eval_accuracy': 0.034, 'eval_runtime': 18.3037, 'eval_samples_per_second': 54.634, 'eval_steps_per_second': 1.748}
eval_peft_vk
 {'eval_loss': 1.7993906736373901, 'eval_accuracy': 0.034, 'eval_runtime': 18.0723, 'eval_samples_per_second': 55.333, 'eval_steps_per_second': 1.771}
eval_peft_q
 {'eval_loss': 1.7993906736373901, 'eval_accuracy': 0.034, 'eval_runtime': 18.876, 'eval_samples_per_second': 52.977, 'eval_steps_per_second': 1.695}
Base accuracy:  0.034 
PEFT_vk accuracy:  0.034 
PEFT_q accuracy:  0.034
